In [3]:
# pip install openai
# pip install streamlit
# pip install dotenv

In [4]:
import streamlit as st

from openai import OpenAI
from dotenv import load_dotenv
import os

import joblib
import pandas as pd
import numpy as np

In [5]:
# Chat GPT API 설정
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = OPENAI_API_KEY)

In [ ]:
tools_presume = [
    {
        "type" : "function",
        "name" : "get_today_date",
        "description" : "오늘 날짜를 YYYY-MM-DD형식으로 출력",
        "parameters" : {
            "type" : "object",
            "properties" : {},
            "required" : []
        }
    }
]

In [ ]:
# 날짜, 홈팀, 어웨이팀을 넣으면 과거의 경기를 예측해준다.
def presume_match_past(MatchDate, HomeTeam, AwayTeam) :
    # 예측 모델 및 데이터 불러오기
    model = joblib.load('./model_final.pkl')
    data = pd.read_csv('./data_final.csv')

    # MatchDate를 datetime 형식으로 변환
    data['MatchDate'] = pd.to_datetime(data['MatchDate'])
    MatchDate = pd.to_datetime(MatchDate)

    data_for_predict = data[(data['MatchDate'] == MatchDate) & (data['HomeTeam'] == HomeTeam) & (data['AwayTeam'] == AwayTeam)]
    data_for_predict = data_for_predict.drop(columns=['MatchDate'])

    # 예측을 위한 데이터가 있는지 확인
    if data_for_predict.empty :
        # 해당 날짜에 대한 경기 데이터가 없는 경우
        print("해당 날짜에 대한 경기 데이터가 없습니다 ㅠㅠ")
        return 

    else :
        # 데이터 전처리(로그 스케일)
        log_columns = ['HomeElo', 'AwayElo'] # 로그 변환할 columns
        data_for_predict[log_columns] = np.log1p(data_for_predict[log_columns])

        prob = model.predict_proba(data_for_predict)
        

    print(prob)

    # 확률을 반환 0: 홈팀 승리 확률, 1: 어웨이팀 승리 확률
    return prob


# 홈팀, 어웨이팀을 넣으면 가장 최근 경기 결과를 토대로 미래 경기를 예측한다.
def presume_match(HomeTeam, AwayTeam) :
    data = pd.read_csv('./data_final.csv')
    model = joblib.load('./model_final.pkl')

    # 2) 홈·어웨이 조건으로 필터링
    sub = data[(data['HomeTeam'] == HomeTeam) & (data['AwayTeam'] == AwayTeam)]
    if sub.empty:
        print("두 팀의 맞대결 기록이 없습니다.")
        return None

    # 3) 가장 최근 경기 한 건 선택
    data_for_predict = sub.sort_values('MatchDate', ascending=False).iloc[0:1]  # DataFrame 형태 유지


    # 예측을 위한 데이터가 있는지 확인
    if data_for_predict.empty :
        # 해당 날짜에 대한 경기 데이터가 없는 경우
        print("해당 날짜에 대한 경기 데이터가 없습니다 ㅠㅠ")
        return 

    else :
        # 데이터 전처리(로그 스케일)
        log_columns = ['HomeElo', 'AwayElo'] # 로그 변환할 columns
        data_for_predict[log_columns] = np.log1p(data_for_predict[log_columns])

        prob = model.predict_proba(data_for_predict)

    # 확률을 반환 0: 홈팀 승리 확률, 1: 어웨이팀 승리 확률
    return prob



In [14]:
# 2020-12-27 , Wolves ,Tottenham
presume_match_past('2020-12-27', 'Wolves', 'Tottenham')

[[0.53849707 0.46150293]]


array([[0.53849707, 0.46150293]])

In [13]:
presume_match('Wolves', 'Brentford')

[[0.41121914 0.58878086]]


array([[0.41121914, 0.58878086]])